<a href="https://colab.research.google.com/github/ShawnTang1709/tibameHW/blob/main/20250114.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#HTTP代碼
```python
2開頭: 成功
3開頭: 轉址
#要求網址已經搬家，雖輸入舊網址，但會自動替換成新網址
4開頭: 錯誤
404: Not Found(最常見:錯誤的網址)
403: Forbidden
-原因1:IP位置被BAN (ip ban)
   (通常因為有數次的惡意行為(短時間太多請求...)
   (解法:等待/換ip)
-原因2:行為模式不像瀏覽器
   (解法:學得像一點(headers完善))
```

#Header
```python
介紹:
在送出網址時，其實不只送出網址，真正送出的是:網址+額外訊息(url + headers)。
在得到回應時，其實不只得到回應，真正得到的是:回應+額外訊息(response + headers)

```

#處理檔案
```python
1.純文字(txt/csv): open(....,"r"or"w", encoding="utf-8")
2.非純文字(圖片/pdf/doc...): open(..., "rb"or"wb")
```

In [10]:
import urllib.request as req
import bs4 as bs
import os

url = "https://www.ptt.cc/bbs/Beauty/M.1736653956.A.DCC.html"
r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36")

#創建資料夾
filename = "pic" + ".json"
dirname = "pic"
if not os.path.exists(filename):
  os.makedirs(filename)

response = req.urlopen(r)
html = bs.BeautifulSoup(response)

allow_subname = ["jpg", "jpeg", "png", "gif"]
links = html.find_all("a")
for l in links:
  href = l["href"]
  subname = href.split(".")[-1]
  if subname.lower() in allow_subname:
    r = req.Request(href)
    r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36")
    img = req.urlopen(r)
    #open一個檔案來儲存
    fname = href.split("/")[-1]
    f = open(fname, "wb")
    f.write(img.read())
    f.close
    # print(img.read())
    # print("-" * 30)

In [40]:
import urllib.request as req
import bs4 as bs
import os
import json

url = "https://www.ptt.cc/bbs/Beauty/M.1736653956.A.DCC.html"
#建立檔案的檔案名稱
url_split = url.split( "/")
fn = url_split[-1] + ".json"
dirname = url_split[-2]
#如果資料夾不存在，則創建資料夾
if not os.path.exists(dirname):
  os.makedirs(dirname)
fpath = dirname + "/" + fn

#取得網址
r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36")
response = req.urlopen(r)
html = bs.BeautifulSoup(response)
info = html.find_all("span", {"class":"article-meta-value"})

#找到名字/暱稱
name = info[0].text
if "(" in name:
  id = name.split("(")
  name = id[0].strip()
  nickname = id[-1].replace(")", "")
else:
  nickname = ""
#看板
board = info[1].text
#找到標題
title = info[2].text
#找到分類
if title[0] == "[":
  title_split = title.split("]")
  category = title_split[0].replace("[", "")
  title = title_split[-1].strip()
else:
  category = ""
#找到時間
time = info[3].text
# #看板
# block = html.find("a", {"class":"board"}).text

# print("作者:", name)
# print("暱稱:", nickname)
# print("看板:", board)
# print("分類:", category)
# print("標題:", title)
# print("時間:", time)
data = {
    "id": name,
    "nickname": nickname,
    "board": board,
    "category": category,
    "title": title,
    "time": time,
    "push": []
}
#推噓文(使用Json存檔)
push_list = html.find_all("div", {"class":"push"})
for push in push_list:
  push_meta = push.find_all("span")
  push_type = push_meta[0].text.strip()
  push_name = push_meta[1].text.strip()
  push_content = push_meta[2].text.replace(":", "").strip()
  push_ip_date = push_meta[3].text.strip()
  (push_type, push_name, push_content, push_ip_date)
  push_data = {
      "type": push_type,
      "push_id": push_name,
      "content": push_content,
      "ip_date": push_ip_date
      }
  data["push"].append(push_data)
  # print(push_data)
  #把json寫入檔案裡
  f = open(fpath, "w", encoding="utf-8")
  #之前是使用f.write，現在使用json.dump幫我做更好的寫入
  json.dump(data, f, ensure_ascii = False, indent=4)
